<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/DOC_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install pdfminer.six

## **UNSUPERVISED LEARNING**

In [14]:
# Necessary imports
import os
from pdfminer.high_level import extract_text
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np


# Define the path to your folder containing the PDFs on Google Drive
pdf_directory = '/content/drive/MyDrive/docs'  # Replace 'your_folder_name' with the appropriate folder name
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# 1. Extract text from PDFs
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

texts = [extract_text_from_pdf(os.path.join(pdf_directory, pdf_file)) for pdf_file in pdf_files]

# 2. Convert text into vectors
texts_tokenized = [text.split() for text in texts]
model = Word2Vec(sentences=texts_tokenized, vector_size=100, window=5, min_count=1, workers=4)
word_vectors = [model.wv[text] for text in texts_tokenized]

# Assuming the documents are long, take an average of word vectors for simplicity
doc_vectors = [sum(vec)/len(vec) if len(vec) != 0 else np.zeros(100) for vec in word_vectors]

# Convert to numpy array
doc_vectors = np.array(doc_vectors, dtype=np.float32)

# 3. Train an autoencoder
encoding_dim = 32
input_vec = Input(shape=(100,))
encoded = Dense(encoding_dim, activation='relu')(input_vec)
decoded = Dense(100, activation='sigmoid')(encoded)

autoencoder = Model(input_vec, decoded)
encoder = Model(input_vec, encoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Adjust batch size
batch_size = min(256, len(doc_vectors))
autoencoder.fit(doc_vectors, doc_vectors, epochs=50, batch_size=batch_size, shuffle=True)

encoded_docs = encoder.predict(doc_vectors)

# 4. Apply clustering
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(encoded_docs)

print(clusters)


Epoch 1/50
1/1 [==============================] - 1s 726ms/step - loss: 0.6930
Epoch 2/50
1/1 [==============================] - 0s 14ms/step - loss: 0.6930
Epoch 3/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6930
Epoch 4/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6930
Epoch 5/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6930
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6930
Epoch 7/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6930
Epoch 8/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6930
Epoch 9/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6930
Epoch 10/50
1/1 [==============================] - 0s 14ms/step - loss: 0.6930
Epoch 11/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6930
Epoch 12/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6930
Epoch 13/50
1/1 [==============================] - 0s 10ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[1 2 2 2 2 1 1 2 2 2 1 2 1 1 2 1 2 1 1 2 2 1 2 0 1 1 2 2 2 0 2 0 0 0 2 0 0
 0 0 2 2 2 2 0 0 0 2 2 0 2 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0]


In [15]:
clustered_pdfs = {i: [] for i in range(3)}  # Initialize a dictionary with 3 keys for the 3 clusters

for pdf_file, cluster_label in zip(pdf_files, clusters):
    clustered_pdfs[cluster_label].append(pdf_file)

# Now, you can print the PDFs in each cluster:
for cluster_label, pdf_list in clustered_pdfs.items():
    print(f"Cluster {cluster_label}:")
    for pdf in pdf_list:
        print(f"  - {pdf}")
    print("\n")


Cluster 0:
  - doc_000825.searchable.pdf
  - doc_000080.searchable.pdf
  - doc_000091.searchable.pdf
  - doc_000101.searchable.pdf
  - doc_000169.searchable.pdf
  - doc_000175.searchable.pdf
  - doc_000191.searchable.pdf
  - doc_000248.searchable.pdf
  - doc_000281.searchable.pdf
  - doc_000369.searchable.pdf
  - doc_000375.searchable.pdf
  - doc_000411.searchable.pdf
  - doc_000809.searchable.pdf
  - doc_000016.searchable.pdf
  - doc_000045.searchable.pdf
  - doc_000050.searchable.pdf
  - doc_000121.searchable.pdf
  - doc_000130-2.searchable.pdf
  - doc_000171.searchable.pdf
  - doc_000240.searchable.pdf
  - doc_000250.searchable.pdf
  - doc_000298.searchable.pdf
  - doc_000309.searchable.pdf
  - doc_000334.searchable.pdf
  - doc_000345.searchable.pdf
  - doc_000349.searchable.pdf
  - doc_000845.searchable.pdf
  - doc_000864.searchable.pdf
  - doc_000891.searchable.pdf
  - doc_000950.searchable.pdf


Cluster 1:
  - doc_000278.searchable.pdf
  - doc_000347.searchable.pdf
  - doc_000363

In [16]:
import os

# Path to the main directory containing the subfolders
ground_truth_dir = '/content/drive/MyDrive/dataset Classification'  # Replace with the correct path

# Get list of subfolders (assuming they represent the cluster names)
subfolders = [f.path for f in os.scandir(ground_truth_dir) if f.is_dir()]

# Dictionary to store the ground truth filenames by cluster
ground_truth_pdfs = {}

for subfolder in subfolders:
    cluster_name = os.path.basename(subfolder)
    ground_truth_pdfs[cluster_name] = [f for f in os.listdir(subfolder) if f.endswith('.pdf')]

# Compare predicted clusters with ground truth
correct_count = 0
total_count = len(pdf_files)

# For each predicted cluster, find the ground truth cluster with maximum overlap
for cluster_label, pdf_list in clustered_pdfs.items():
    best_match = None
    best_overlap = 0

    for truth_cluster, truth_pdfs in ground_truth_pdfs.items():
        overlap = len(set(pdf_list).intersection(set(truth_pdfs)))

        if overlap > best_overlap:
            best_overlap = overlap
            best_match = truth_cluster

    correct_count += best_overlap
    print(f"Predicted Cluster {cluster_label} best matches with Ground Truth Cluster {best_match}")

accuracy = correct_count / total_count
print(f"Accuracy based on best match overlap: {accuracy * 100:.2f}%")


Predicted Cluster 0 best matches with Ground Truth Cluster Paper
Predicted Cluster 1 best matches with Ground Truth Cluster Email
Predicted Cluster 2 best matches with Ground Truth Cluster Email
Accuracy based on best match overlap: 60.00%


In [17]:
import os

# Path to the main directory containing the subfolders
ground_truth_dir = '/content/drive/MyDrive/dataset Classification'  # Replace with the correct path

# Get list of subfolders (assuming they represent the cluster names)
subfolders = [f.path for f in os.scandir(ground_truth_dir) if f.is_dir()]

# Dictionary to store the ground truth filenames by cluster
ground_truth_pdfs = {}

for subfolder in subfolders:
    cluster_name = os.path.basename(subfolder)  # Use the subfolder name as the cluster name
    ground_truth_pdfs[cluster_name] = [f for f in os.listdir(subfolder) if f.endswith('.pdf')]

# Compare predicted clusters with ground truth
total_correct_count = 0
total_count = len(pdf_files)

# For each predicted cluster, find the ground truth cluster with maximum overlap
for cluster_label, pdf_list in clustered_pdfs.items():
    best_match = None
    best_overlap = 0
    correct_pdfs = []

    for truth_cluster, truth_pdfs in ground_truth_pdfs.items():
        overlap = len(set(pdf_list).intersection(set(truth_pdfs)))

        if overlap > best_overlap:
            best_overlap = overlap
            best_match = truth_cluster  # This now represents the name of the ground truth subfolder
            correct_pdfs = list(set(pdf_list).intersection(set(truth_pdfs)))

    accuracy_for_cluster = best_overlap / len(pdf_list)
    total_correct_count += best_overlap
    print(f"Predicted Cluster {cluster_label} best matches with Ground Truth Cluster '{best_match}'")
    print(f"Accuracy for Predicted Cluster {cluster_label} (matching with '{best_match}'): {accuracy_for_cluster * 100:.2f}%")
    print("Correctly Predicted PDFs for this Cluster:")
    for pdf in correct_pdfs:
        print(f"  - {pdf}")
    print("\n")

overall_accuracy = total_correct_count / total_count
print(f"Overall Accuracy based on best match overlap: {overall_accuracy * 100:.2f}%")


Predicted Cluster 0 best matches with Ground Truth Cluster 'Paper'
Accuracy for Predicted Cluster 0 (matching with 'Paper'): 56.67%
Correctly Predicted PDFs for this Cluster:
  - doc_000050.searchable.pdf
  - doc_000309.searchable.pdf
  - doc_000240.searchable.pdf
  - doc_000250.searchable.pdf
  - doc_000171.searchable.pdf
  - doc_000845.searchable.pdf
  - doc_000121.searchable.pdf
  - doc_000345.searchable.pdf
  - doc_000298.searchable.pdf
  - doc_000891.searchable.pdf
  - doc_000130-2.searchable.pdf
  - doc_000045.searchable.pdf
  - doc_000349.searchable.pdf
  - doc_000864.searchable.pdf
  - doc_000334.searchable.pdf
  - doc_000016.searchable.pdf
  - doc_000950.searchable.pdf


Predicted Cluster 1 best matches with Ground Truth Cluster 'Email'
Accuracy for Predicted Cluster 1 (matching with 'Email'): 100.00%
Correctly Predicted PDFs for this Cluster:
  - doc_000483.searchable.pdf
  - doc_000278.searchable.pdf
  - doc_000745.searchable.pdf
  - doc_000840.searchable.pdf
  - doc_000363.

## **USING SUPERVISED LEARNING**

In [20]:
import pdfminer
from pdfminer.high_level import extract_text

def extract_pdf_content(pdf_file):
    try:
        return extract_text(pdf_file)
    except Exception as e:
        print(f"Error processing PDF file {pdf_file}: {e}")
        return ""


In [26]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pdfminer
from pdfminer.high_level import extract_text

# Define the function to extract text from PDF files
def extract_pdf_content(pdf_file):
    try:
        return extract_text(pdf_file)
    except Exception as e:
        print(f"Error processing PDF file {pdf_file}: {e}")
        return ""

# Path to the main directory containing the subfolders
ground_truth_dir = '/content/drive/MyDrive/dataset Classification'

# Define subfolder names
clusters_names = ['Resume', 'Email', 'Paper']

# Fetch documents from the folders
documents = []
labels = []

for idx, cluster in enumerate(clusters_names):
    folder_path = os.path.join(ground_truth_dir, cluster)
    for file in os.listdir(folder_path):
        if file.endswith('.pdf'):
            with open(os.path.join(folder_path, file), 'rb') as f:
                # Convert PDF to text
                content = extract_pdf_content(f)
                documents.append(content)
                labels.append(idx)  # Use idx as label

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(documents, labels, test_size=0.2, random_state=42)

# Text Vectorization - TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

# Training a Logistic Regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_transformed, y_train)

# Predictions
y_pred = clf.predict(X_test_transformed)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=clusters_names))


Accuracy: 0.9285714285714286
              precision    recall  f1-score   support

      Resume       1.00      0.88      0.93         8
       Email       0.80      1.00      0.89         4
       Paper       1.00      1.00      1.00         2

    accuracy                           0.93        14
   macro avg       0.93      0.96      0.94        14
weighted avg       0.94      0.93      0.93        14



In [27]:
# Assuming the previous code has been run and we have the `vectorizer` and `clf` objects instantiated

# Path to the directory containing the new PDFs
test_data_dir = '/content/drive/MyDrive/test'

# Fetch documents from the folder
test_documents = []
test_file_names = []

for file in os.listdir(test_data_dir):
    if file.endswith('.pdf'):
        with open(os.path.join(test_data_dir, file), 'rb') as f:
            # Convert PDF to text
            content = extract_pdf_content(f)
            test_documents.append(content)
            test_file_names.append(file)

# Transform the test documents using the previously trained vectorizer
X_test_unknown = vectorizer.transform(test_documents)

# Predictions
y_pred_unknown = clf.predict(X_test_unknown)

# Get prediction probabilities
y_pred_prob_unknown = clf.predict_proba(X_test_unknown)

# Extract the certainty (probability) of the predicted class
certainty_levels = [prob[pred] for prob, pred in zip(y_pred_prob_unknown, y_pred_unknown)]

# Display predictions and certainty levels
for file_name, pred, certainty in zip(test_file_names, y_pred_unknown, certainty_levels):
    print(f"{file_name}: {clusters_names[pred]} with certainty: {certainty*100:.2f}%")


doc_000575-2.searchable.pdf: Resume with certainty: 52.56%
doc_000752.searchable.pdf: Resume with certainty: 47.36%
doc_000076.searchable.pdf: Email with certainty: 54.63%
doc_000111.searchable.pdf: Email with certainty: 59.25%
doc_000824.searchable.pdf: Resume with certainty: 58.68%
doc_000089.searchable.pdf: Paper with certainty: 43.43%
doc_000142.searchable.pdf: Email with certainty: 50.03%
doc_000285.searchable.pdf: Paper with certainty: 45.34%


All the predictions are CORRECT, yet certainty % are average

### **USE DIFFERENT ML MODELS**

In [31]:
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pdfminer
from pdfminer.high_level import extract_text
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# PDF Extraction function
def extract_pdf_content(pdf_path):
    return extract_text(pdf_path)

# Load Data from Google Drive
data_dir = '/content/drive/MyDrive/dataset Classification'
clusters_names = ['Resume', 'Email', 'Paper']
documents = []
labels = []

for idx, cluster_name in enumerate(clusters_names):
    cluster_dir = os.path.join(data_dir, cluster_name)
    for file_name in os.listdir(cluster_dir):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(cluster_dir, file_name)
            content = extract_pdf_content(file_path)
            documents.append(content)
            labels.append(idx)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(documents, labels, test_size=0.2, random_state=42)

# Text Vectorization - TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1,2))
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Gradient Boosting Machines': GradientBoostingClassifier(n_estimators=100)
}

# Tokenization for Deep Learning
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 250
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train)
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train_padded = np.array(pad_sequences(X_train_tokenized, maxlen=MAX_SEQUENCE_LENGTH))
X_test_padded = np.array(pad_sequences(X_test_tokenized, maxlen=MAX_SEQUENCE_LENGTH))
y_train = np.array(y_train)

# LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(MAX_NUM_WORDS, 128))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))

lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.1)
lstm_preds = lstm_model.predict(X_test_padded).argmax(axis=1)

# Evaluate traditional models
for name, model in models.items():
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_test_transformed)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"{name}: F1 Score = {f1:.2f}")

# Evaluate LSTM
f1_lstm = f1_score(y_test, lstm_preds, average='weighted')
print(f"LSTM: F1 Score = {f1_lstm:.2f}")

Epoch 1/5
2/2 [==============================] - 7s 1s/step - loss: 1.1002 - accuracy: 0.2800 - val_loss: 1.1057 - val_accuracy: 0.1667
Epoch 2/5
2/2 [==============================] - 1s 703ms/step - loss: 1.0841 - accuracy: 0.5400 - val_loss: 1.1112 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 1s 707ms/step - loss: 1.0659 - accuracy: 0.6400 - val_loss: 1.1225 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 1s 683ms/step - loss: 1.0419 - accuracy: 0.6400 - val_loss: 1.1456 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 345ms/step
Logistic Regression: F1 Score = 0.93
Random Forest: F1 Score = 0.86
Gradient Boosting Machines: F1 Score = 0.78
LSTM: F1 Score = 0.13


# Logistic Regression: F1 Score = 0.93
# Random Forest: F1 Score = 0.86
# Gradient Boosting Machines: F1 Score = 0.78
# LSTM: F1 Score = 0.13

## **BERT CLASSIFICATION**

In [32]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00


In [33]:
import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from sklearn.model_selection import train_test_split


# Define the paths to your data
base_path = '/content/drive/MyDrive/dataset Classification'
data_folders = ['Resume', 'Email', 'Paper']

texts = []
labels = []

for idx, folder_name in enumerate(data_folders):
    folder_path = os.path.join(base_path, folder_name)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()
            texts.append(content)
            labels.append(idx)

# Split data into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# BERT processing and fine-tuning

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def convert_data_to_examples(texts, labels):
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=text, text_b=None, label=label)
        )
    return InputExamples

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )
        input_ids, token_type_ids, attention_mask = (
            input_dict["input_ids"],
            input_dict["token_type_ids"],
            input_dict["attention_mask"],
        )
        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

# Convert the data
train_examples = convert_data_to_examples(X_train, y_train)
test_examples = convert_data_to_examples(X_test, y_test)

train_data = convert_examples_to_tf_dataset(train_examples, tokenizer).shuffle(100).batch(32).repeat(2)
test_data = convert_examples_to_tf_dataset(test_examples, tokenizer).batch(32)

# Fine-tune BERT for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.fit(train_data, epochs=2, validation_data=test_data)

# Evaluate on test data
results = model.evaluate(test_data)
print("Test accuracy: ", results[1])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
4/4 [==============================] - 256s 49s/step - loss: 1.1114 - accuracy: 0.3125 - val_loss: 1.2801 - val_accuracy: 0.2857
Epoch 2/2
1/1 [==============================] - 9s 9s/step - loss: 1.2686 - accuracy: 0.2857
Test accuracy:  0.2857142984867096
